In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [2]:
%pip install 'mlforecast[lag_transforms]'

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pathlib import Path

import lightgbm as lgb
import numpy as np
import polars as pl
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean, SeasonalRollingMean

In [5]:
calendar = "/kaggle/input/m5-forecasting-accuracy/calendar.csv"
train_data = "/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv"
sell_prices = "/kaggle/input/m5-forecasting-accuracy/sell_prices.csv"
submission = "/kaggle/input/m5-forecasting-accuracy/sample_submission.csv"

In [6]:
calendar_data = pd.read_csv(calendar)
event_cols = ['event_name_1','event_type_1','event_name_2', 'event_type_2']
calendar_data[event_cols] = calendar_data[event_cols].fillna(np.nan)

In [7]:
price_data = pd.read_csv(sell_prices)

In [8]:
sales_data = pd.read_csv(train_data)

In [9]:
data_frame = pd.melt(frame=sales_data, 
             id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
             var_name="d", value_name="sales")


In [10]:
data_frame.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [11]:
data_frame['d'] = data_frame["d"].apply(lambda x: int(x[2:])).astype(np.int16)
dates = sorted(data_frame['d'].unique())
data_frame = data_frame.sort_values(by='d')
data_frame['sales_new'] = data_frame['sales'].gt(0).cummax()

# Select rows based on the condition
without_leading_zeros = data_frame[data_frame['sales_new']].reset_index(drop=True)['sales']
above_min_date = (data_frame['d'] >= dates[-426]) & (data_frame['d'] <= dates[-29])
above_min_date_test = data_frame['d'] >= dates[-29]
total_above_min_date = data_frame['d'] >= dates[-700]
keep_mask = without_leading_zeros & above_min_date
mask_test = without_leading_zeros & above_min_date_test
mask_total = total_above_min_date & total_above_min_date
data_frame_train = data_frame[keep_mask]
data_frame_test = data_frame[mask_test]
data_frame_total = data_frame[mask_total]

/tmp/ipykernel_1265/4283002137.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_frame_train = data_frame[keep_mask]
/tmp/ipykernel_1265/4283002137.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_frame_test = data_frame[mask_test]


In [12]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [13]:
data_frame_total = downcast(data_frame_total)
calendar_data = downcast(calendar_data)
price_data = downcast(price_data)

/tmp/ipykernel_1265/2961721585.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype('category')
/tmp/ipykernel_1265/2961721585.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype('category')
/tmp/ipykernel_1265/2961721585.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [14]:
del data_frame

In [15]:
calendar_data['d'] = calendar_data["d"].apply(lambda x: int(x[2:])).astype(np.int16)

In [16]:
price_data.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [17]:
data_frame_total = pd.merge(data_frame_total, calendar_data, on=['d'])
data_frame_total = pd.merge(data_frame_total, price_data, on=['store_id', 'item_id', 'wm_yr_wk'])
last_wmyrwk_total = data_frame_total['wm_yr_wk'].max()
last_date_total = data_frame_total['date'].max()
data_frame_total = data_frame_total.drop(columns=['wm_yr_wk'], axis=1)

# build future X
future_cal_total = calendar_data[calendar_data['date'] > last_date_total]
future_prices_total = price_data[price_data['wm_yr_wk'] >= last_wmyrwk_total]


In [18]:
future_prices_total.head()

,store_id,item_id,wm_yr_wk,sell_price
149,CA_1,HOBBIES_1_001,11617,8.382812
150,CA_1,HOBBIES_1_001,11618,8.382812
151,CA_1,HOBBIES_1_001,11619,8.382812
152,CA_1,HOBBIES_1_001,11620,8.382812
153,CA_1,HOBBIES_1_001,11621,8.382812


In [19]:
future_prices_total['id'] = future_prices_total['item_id'].astype(str) + '_' + future_prices_total['store_id'].astype(str) + '_evaluation'

/tmp/ipykernel_1265/2544930070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_prices_total['id'] = future_prices_total['item_id'].astype(str) + '_' + future_prices_total['store_id'].astype(str) + '_evaluation'


In [20]:
future_prices_total.head()

,store_id,item_id,wm_yr_wk,sell_price,id
149,CA_1,HOBBIES_1_001,11617,8.382812,HOBBIES_1_001_CA_1_evaluation
150,CA_1,HOBBIES_1_001,11618,8.382812,HOBBIES_1_001_CA_1_evaluation
151,CA_1,HOBBIES_1_001,11619,8.382812,HOBBIES_1_001_CA_1_evaluation
152,CA_1,HOBBIES_1_001,11620,8.382812,HOBBIES_1_001_CA_1_evaluation
153,CA_1,HOBBIES_1_001,11621,8.382812,HOBBIES_1_001_CA_1_evaluation


In [21]:
X_data_frame_total_total = pd.merge(future_prices_total, future_cal_total, on='wm_yr_wk')

In [22]:
X_data_frame_total_total = X_data_frame_total_total.drop(columns=['store_id', 'item_id', 'wm_yr_wk', 'd'], axis=1)

In [23]:
mlfmodel = MLForecast(
    models=[],
    freq='1d',
    lags=[7,14,21,28,29,30,31,32,33,34,35],
    lag_transforms = {
        1 :  [ExpandingMean()],
        7:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        14:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        21:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        28:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        35:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)]
    },
    date_features=['year', 'month', 'day', 'weekday', 'quarter', 'week'],    
    num_threads=10,
)

In [24]:
data_frame_total.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,sales_new,date,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_2_206_TX_3_evaluation,FOODS_2_206,FOODS_2,FOODS,TX_3,TX,1242,1,True,2014-06-23,...,6,2014,NaN,NaN,NaN,NaN,0,0,0,9.867188
1,FOODS_2_206_TX_3_evaluation,FOODS_2_206,FOODS_2,FOODS,TX_3,TX,1243,0,True,2014-06-24,...,6,2014,NaN,NaN,NaN,NaN,0,0,0,9.867188
2,FOODS_2_206_TX_3_evaluation,FOODS_2_206,FOODS_2,FOODS,TX_3,TX,1244,0,True,2014-06-25,...,6,2014,NaN,NaN,NaN,NaN,0,0,0,9.867188
3,FOODS_2_206_TX_3_evaluation,FOODS_2_206,FOODS_2,FOODS,TX_3,TX,1245,0,True,2014-06-26,...,6,2014,NaN,NaN,NaN,NaN,0,0,0,9.867188
4,FOODS_2_206_TX_3_evaluation,FOODS_2_206,FOODS_2,FOODS,TX_3,TX,1246,0,True,2014-06-27,...,6,2014,NaN,NaN,NaN,NaN,0,0,0,9.867188


In [ ]:
categoricals = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
X, y = mlfmodel.preprocess(
    data_frame_total,
    id_col='id',
    time_col='date',
    target_col='sales',
    static_features=categoricals,
    return_X_y=True,    
    as_numpy=True,
)

In [ ]:
del calendar_datada
del prices_data
del sales_data


In [ ]:
mlfmodel.ts.features_order_

In [ ]:
model_params = {"objective" : "tweedie",
        'boosting_type': 'gbdt',
        'learning_rate': 0.034,
        'max_depth' : 135,
        'num_leaves': 88,
        'n_estimators': 1000,
        'force_row_wise':True,
        'lambda_l2':1
}

In [ ]:
lgb_model = lgb.LGBMRegressor(**model_params)

In [ ]:
params = {
    'objective': 'reg:squarederror',  
    'colsample_bytree': 0.3,          
    'learning_rate': 0.1,             
    'max_depth': 5,                   
    'alpha': 10,                      
    'n_estimators': 100               
}


In [ ]:
import xgboost as xgb


xgb_model = xgb.XGBRegressor(**params)

In [ ]:
lgb_model.fit(X, y, feature_name=mlfmodel.ts.features_order_, categorical_feature=categoricals)

In [ ]:
import matplotlib.pyplot as plt

lgb.plot_importance(lgb_model, max_num_features=20, figsize=(10, 6), importance_type='split')  # Adjust max_num_features as needed

plt.title('Feature Importance')
plt.show()

In [ ]:
xgb_model.fit(X, y)

In [ ]:
mlfmodel.models_ = {'LGBMRegressor': lgb_model, 'XGBRegressor': xgb_model}
%time preds_final = mlfmodel.predict(28, X_df=X_data_frame_total_total)

In [ ]:
wide_lgb = preds_final.pivot(values='LGBMRegressor', index='id', columns='date')
wide_lgb.columns = ['id'] + [f'F{i+1}' for i in range(28)]


In [ ]:
wide_xgb = preds_final.pivot(values='XGBRegressor', index='id', columns='date')
wide_xgb.columns = ['id'] + [f'F{i+1}' for i in range(28)]

In [ ]:
sub_lgb = pd.concat([wide_lgb, wide_lgb.assign(id=wide_lgb['id'].astype(str).str.replace('evaluation', 'validation'))])

# Write to CSV
sub_lgb.to_csv('submission_lgb.csv', index=False)

In [ ]:
sub_xgb = pd.concat([wide_xgb, wide_xgb.assign(id=wide_lgb['id'].astype(str).str.replace('evaluation', 'validation'))])

# Write to CSV
sub_xgb.to_csv('submission_lgb.csv', index=False)

In [ ]:
sub_lgb.head()

In [ ]:
sub_xgb.head()

In [ ]:
index_df = sub_lgb['id']

In [ ]:
sub_lgb = sub_lgb.drop(['id'], axis =1)
sub_xgb = sub_xgb.drop(['id'], axis = 1)


In [ ]:
average_values = (sub_xgb + sub_lgb) / 2

In [ ]:
average_values['id'] = index_df

In [ ]:
average_values

In [ ]:
average_values.to_csv('submission_ensambled_xgb_lsg.csv')